In [1]:
#Importación de librearías necesarias
import pandas as pd
from datetime import datetime
import socket
import numpy as np
import math
import time
import pickle  #Para guardar archivos
import os
os.environ['OMP_NUM_THREADS'] = '1'

from pympler import asizeof #Para liberar memoria
import gc

from matplotlib import pyplot as plt

In [35]:
#Path general de archivos
if socket.gethostname()=='LAPTOP-PUSGG08B': #Ip de la laptop
    ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/Data/TFM/"
    r_ruta = "E:/Cristian Guatemal/Master/Big Data y Ciencia de Datos/VIU_TFM/RData/TFM/"
elif socket.gethostname()=='PCUIOMTDAIE6382': #Ip del working
    ruta =   "D:/Master/Big_Data_Ciencia_Datos/VIU_TFM/Data/TFM/"
    r_ruta = "D:/Master/Big_Data_Ciencia_Datos/VIU_TFM/RData/TFM/"

# Ruta del archivo de pensionistas de vejez
ruta_vj = ruta + 'POB_VEJ_CD656_NEW.dsv'
# Ruta del archivo de pensionistas de vejez
ruta_inv = ruta + 'POB_INV_CD656_NEW.dsv'
# Ruta del archivo de pensionistas de vejez
ruta_dis = ruta + 'POB_DIS_CD656_NEW.dsv'

# Ruta del archivo de historia laboral de pensionistas
ruta_afi = ruta + 'APORTES_CD656_new.dsv'

In [ ]:
# Cargar archivo------------------------------------------------------------------------------------------------------------
##Archivo con cluster jerarquico
print('Se cargan los archivos necesarios')
inicio = time.time()
directorio = r_ruta
nombre_archivo = 'viu_cj_data_ati.pkl'
ruta_archivo = os.path.join(directorio, nombre_archivo)

with open( ruta_archivo, 'rb') as archivo:
    data_cj = pickle.load( archivo ) 
    
##Archivo con kmean
directorio = r_ruta
nombre_archivo = 'viu_kmean_data_ati.pkl'
ruta_archivo = os.path.join(directorio, nombre_archivo)

with open( ruta_archivo, 'rb') as archivo:
    data_km = pickle.load( archivo ) 
    
##Archivo con dbscan
directorio = r_ruta
nombre_archivo = 'viu_db_data_ati.pkl'
ruta_archivo = os.path.join(directorio, nombre_archivo)

with open( ruta_archivo, 'rb') as archivo:
    data_db = pickle.load( archivo ) 
    
fin = time.time()  
tm = fin-inicio
print('Tiempo de ejecución es:',tm//3600,'horas con',tm%3600//60 ,'minutos y',tm%60, 'segundos' )

In [ ]:
#Data cluster jerarquico
data_cj.head(20)

In [ ]:
#Data kmean
data_km.head(20)

In [ ]:
#Data dbscan
data_db.head(20)

In [ ]:
#Se verifica que tengan el mismo indice

# data_cj[data_cj['INDICE']!=data_km['INDICE']] #Todo igual
# data_cj[data_cj['INDICE']!=data_db['INDICE']] #Todo igual
# data_km[data_km['INDICE']!=data_db['INDICE']] #Todo igual

#Se verifica que tengan el mismo salario
# data_cj[data_cj['SALARIO']!=data_km['SALARIO']] #Todo igual
# data_cj[data_cj['SALARIO']!=data_db['SALARIO']] #Todo igual
# data_km[data_km['SALARIO']!=data_db['SALARIO']] #Todo igual


# data1.set_index('INDICE', inplace=True )
# cedul = list(dic_sim.keys())  #total de 111903
# filtro = data1[ data1['CEDULA_COD'].isin(cedul )] # 111903 cedulas
# indi = filtro.index
# ml.loc[ indi, 'ATI_DB_AS'] = filtro['ATI_DB_AS']


In [ ]:
data_cj.columns

In [ ]:
data_km.columns

In [ ]:
data_db.columns

In [ ]:
#Se une los dataframe en un solo archivo
print('Se unen los archivos según la variable de atipicos')
inicio = time.time()

val = ['CEDULA_COD', 'ANIO', 'MES', 'SALARIO', 'SALARIO_SECTOR', 'SECTOR_A',
       'NUM_SEC_MES', '%_NUM_SECTOR', 'GRUPO', 'SAL_PROM_GRUPO', 'GRUPO_SEL',
       'INI_CAL', 'FIN_CAL', 'BASE_CAL', 'SBU', 'ID_SBU', 'INDICE', 'LS1',
       'SAL_PROM1', 'LS2', 'SAL_PROM2', 'LS_MS', 'ATI_CJ']

data_ati = pd.concat( [ data_cj[val], data_km['ATI_KM'], data_db['ATI_DB'] ], axis = 1 )

fin = time.time()  
tm = fin-inicio
print('Tiempo de ejecución es:',tm//3600,'horas con',tm%3600//60 ,'minutos y',tm%60, 'segundos' )

del data_cj, data_km, data_db
gc.collect()

In [ ]:
#Se guardan los resultados 
nombre_archivo = 'viu_data_ati_union.pkl'
# Ruta completa del archivo
ruta_archivo = os.path.join(directorio, nombre_archivo)
# Guardar los objetos en el archivo
with open(ruta_archivo, 'wb') as archivo:
    pickle.dump(data_ati, archivo) 

In [ ]:
# Cargar archivo------------------------------------------------------------------------------------------------------------
directorio = r_ruta
nombre_archivo = 'viu_data_ati_union.pkl'
ruta_archivo = os.path.join(directorio, nombre_archivo)

with open( ruta_archivo, 'rb') as archivo:
    data_ati= pickle.load( archivo ) 

In [ ]:
print('Cedulas con al menos un atipico por el CJ', data_ati[data_ati['ATI_CJ']==1]['CEDULA_COD'].nunique()) #171158
print('Cedulas con al menos un atipico por el KM', data_ati[data_ati['ATI_KM']==1]['CEDULA_COD'].nunique()) #171158
print('Cedulas con al menos un atipico por el DB', data_ati[data_ati['ATI_DB']==1]['CEDULA_COD'].nunique()) # 191935

In [5]:
print( data_ati[ (data_ati['GRUPO_SEL']==1) & (data_ati['ATI_CJ']!=data_ati['ATI_KM'])]['CEDULA_COD'].nunique()) #0
print(data_ati[ (data_ati['GRUPO_SEL']==1) & (data_ati['ATI_CJ']!=data_ati['ATI_DB'])]['CEDULA_COD'].nunique()) #77834
print(data_ati[ (data_ati['GRUPO_SEL']==1) & (data_ati['ATI_KM']!=data_ati['ATI_DB'])]['CEDULA_COD'].nunique()) #77834

0
77834
77834


In [ ]:
data_ati[data_ati['ATI_CJ']==]

In [ ]:
print('Valores unicos en ATI_CJ', data_ati[data_ati['GRUPO_SEL']==1]['ATI_CJ'].unique()) #[0. 1.]
print('Valores unicos en ATI_KM', data_ati[data_ati['GRUPO_SEL']==1]['ATI_KM'].unique()) #[0. 1.]
print('Valores unicos en ATI_DB', data_ati[data_ati['GRUPO_SEL']==1]['ATI_DB'].unique()) #[0. 1.]

In [ ]:
data_ati.columns

In [ ]:
#Calcular la Base de calculo sin considerar los atipicos
def base_cal_sin_ati( data, val ):
    df = data.copy()
    nom = 'BCS_'+ val.split('_')[-1]
    grupo = df[ (df['GRUPO_SEL'] == 1 ) & (df[ val ] != 1)].copy()
    df_prom = grupo.groupby('CEDULA_COD')['SALARIO'].mean()
    data[nom] = data['CEDULA_COD'].map( df_prom ) 

In [ ]:
print('Se corrige la base de calculo sin considerar a los atipicos')
inicio = time.time()
base_cal_sin_ati( data_ati, 'ATI_CJ')
base_cal_sin_ati( data_ati, 'ATI_KM')
base_cal_sin_ati( data_ati, 'ATI_DB')
fin = time.time()  
tm = fin-inicio
print('Tiempo de ejecución es:',tm//3600,'horas con',tm%3600//60 ,'minutos y',tm%60, 'segundos' )

In [ ]:
grouped = data_ati['CEDULA_COD'].drop_duplicates().sample(n=30000, random_state=1)
muestra = data_ati[data_ati['CEDULA_COD'].isin(grouped)]
muestra.to_csv('muestra_data_ati.txt', sep='\t', index=False)# Agrupar por CEDULA_COD

In [ ]:
#Se guardan los resultados 
nombre_archivo = 'viu_data_ati_union_base_corr.pkl'
# Ruta completa del archivo
ruta_archivo = os.path.join(directorio, nombre_archivo)
# Guardar los objetos en el archivo
with open(ruta_archivo, 'wb') as archivo:
    pickle.dump(data_ati, archivo) 

In [4]:
#Se cargan datos para comenzar con la union de los pensionistas de vejez
directorio = r_ruta
nombre_archivo = 'viu_data_ati_union_base_corr.pkl'
ruta_archivo = os.path.join(directorio, nombre_archivo)

with open( ruta_archivo, 'rb') as archivo:
    data_ati= pickle.load( archivo ) 

In [6]:
def graf_metodos(data_i, ced, val, grupo_sel = 0, ati = 'CJ'):
    #Adecuación de la base de datos
    ATI =  'ATI_' + ati
    BCS = 'BCS_' + ati
    
    idx = data_i.columns.get_loc('LS2')
    columnas_requeridas = ['CEDULA_COD', 'ANIO', 'MES', val, 'GRUPO_SEL', 'INDICE', 'BASE_CAL'] + list(data_i.columns[idx:])
    data = data_i[data_i['CEDULA_COD'] == ced][columnas_requeridas]
    
    # Establecer si se quieren todos o solos los 5 mejores años
    if(grupo_sel == 1):
        data = data[data['GRUPO_SEL']==1]
    
    #Fecha para eje x
    anios = data['ANIO'].tolist()
    meses = data['MES'].tolist()
    fechas = [datetime(year=anio, month=mes, day=1) for anio, mes in zip(anios, meses)]
    data['FECHA']= pd.to_datetime(fechas)
    data = data.sort_values(by='FECHA')
    
    #Paleta de colores
    colors = plt.get_cmap('tab20').colors

    # Gráfico
    plt.figure(figsize=(12, 8))  
    marker_dict = {1: 'o', 2: 'x'} 
    
    data_aux = data[data['GRUPO_SEL']==1]
    
    if grupo_sel == 0:  
        Grupo_b= data[(data['GRUPO_SEL'] == 0)]
        plt.scatter(Grupo_b['FECHA'], Grupo_b[val], marker= '^', color='yellow', label='Valores no considerados')
        
        if 0 in (data_aux[ATI].unique()):
            Grupo_a = data_aux[data_aux[ATI] == 0]
            plt.scatter(Grupo_a['FECHA'], Grupo_a[val], marker= marker_dict[1], color='blue', label='Valores No Atípicos : 0')

        if 1 in (data_aux[ATI].unique()):
            Grupo= data_aux[data_aux[ATI] == 1]
            plt.scatter(Grupo['FECHA'], Grupo[val], marker= marker_dict[1], color= 'red', label='Valores Atípicos : 1')
                
    else:
        if 0 in (data[ATI].unique()):
            Grupo_a = data[data[ATI] == 0]
            plt.scatter(Grupo_a['FECHA'], Grupo_a[val], marker= marker_dict[1], color='blue', label='Valores No Atípicos : 0')

        if 1 in (data[ATI].unique()):
            Grupo= data[data[ATI] == 1]
            plt.scatter(Grupo['FECHA'], Grupo[val], marker= marker_dict[1], color= 'red', label='Valores Atípicos : 1')
    
    plt.axhline(y= data['SAL_PROM2'].iloc[0], color= '#1297ff', linestyle='-', label='Promedio Salarios')
    plt.axhline(y=data['LS2'].iloc[0], alpha=1 , color= '#25ce00', linestyle='--', label='LS2')
    plt.axhline(y=data_aux['LS_MS'].iloc[0], alpha=1, color= '#875b20', linestyle='--', label='LS_MS')
    plt.axhline(y=data[BCS].iloc[0], alpha=1, color= '#E800FF', linestyle='-', label='Base de cálculo sin atípicos')
    plt.axhline(y=data['BASE_CAL'].iloc[0], alpha=0.5, color= '#E800FF', linestyle=':', label='Base de cálculo')
    
    # Añadir el número de clusters, EPS, MINPTS
    plt.scatter( data_aux['FECHA'].iloc[0], data_aux[val].iloc[0], facecolors='none', label="Número de clústers: {}".format(int(max(data_aux[ATI].unique()) + 1)))
    
    # Añadir títulos y etiquetas
    plt.title(f"Aportaciones de la cédula: {ced}")
    plt.xlabel('FECHA')
    plt.ylabel(f"{val}")
    
    # Crear la leyenda
    leyenda = plt.legend(loc='upper left', bbox_to_anchor=(1, 1))

    plt.tight_layout()
    
    plt.grid(True)
    plt.show()

In [ ]:
data_ati.head(10)

In [ ]:
graf_metodos( data_ati, 23363, 'SALARIO', grupo_sel = 0, ati = 'DB' )

In [93]:
#Lectura de los pensionistas de vejez
vej = pd.read_csv( ruta_vj, delimiter='\t', encoding='utf-8', decimal='.' )
vej = vej.sort_values( by=["CEDULA_COD"], ascending=[True] )
vej['FECHA_NACIMIENTO'] = pd.to_datetime(vej['FECHA_NACIMIENTO'], format='%d/%m/%Y')
vej['FECHA_MUERTE'] = pd.to_datetime(vej['FECHA_MUERTE'], format='%d/%m/%Y')
vej['FECHA_DERECHO'] = pd.to_datetime(vej['FECHA_DERECHO'], format='%d/%m/%Y')
vej['RANGO_INI_5MEJ'] = pd.to_datetime(vej['RANGO_INI_5MEJ'], format='%d/%m/%Y')
vej['RANGO_FIN_5MEJ'] = pd.to_datetime(vej['RANGO_FIN_5MEJ'], format='%d/%m/%Y')
vej['PRES']='vej'

In [8]:
vej

,CEDULA_COD,SEXO,FECHA_NACIMIENTO,FECHA_MUERTE,FECHA_DERECHO,NUMERO_IMPOSICIONES,COEFICIENTE_REAL,COEFICIENTE_CAL,PROMEDIO_SUELDO_REAL,PROMEDIO_CAL,VALOR_PENSION,PENSION_CAL,ID_PRESTACION,RANGO_INI_5MEJ,RANGO_FIN_5MEJ,N_MESES
16,114,H,1921-07-12,NaT,1994-07-01,429,0.500000,0.8125,0.00,144.9956,56.17,117.808925,1366082,2004-10-31,1991-05-01,581.0
85324,115,H,1940-12-18,NaT,2006-05-01,368,0.758334,0.7500,0.00,364.9616,276.26,273.721200,1304291,2006-04-30,2001-05-01,370.0
679150,117,H,1940-09-01,NaT,2006-03-01,183,0.500000,0.5625,0.00,67.8673,36.64,38.175356,1408488,2006-02-28,2001-03-01,190.0
469427,119,M,1941-06-30,NaT,1997-08-01,421,0.814167,0.8125,0.00,14.4463,29.64,11.737619,1426099,1997-07-31,1992-08-01,421.0
679275,121,M,1924-01-13,2022-01-27,2003-03-01,120,0.500000,0.5000,0.00,40.1640,32.20,20.082000,1403871,2003-02-28,1998-03-01,127.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
341057,24044648,NaN,NaT,NaT,2012-01-01,227,0.612500,0.6000,223.96,NaN,175.20,NaN,2354841,NaT,NaT,NaN
127966,24044649,NaN,NaT,NaT,2014-10-01,180,0.562500,0.5625,0.94,NaN,34.00,NaN,2715528,NaT,NaT,NaN
596796,24179775,NaN,NaT,NaT,2022-07-01,158,0.537500,0.5375,5457.04,NaN,1062.50,NaN,3203256,NaT,NaT,NaN
42607,24207317,NaN,NaT,NaT,2014-01-01,150,0.525000,0.5250,1.29,NaN,163.20,NaN,2386107,NaT,NaT,NaN


In [57]:
print('Número de cedulas de los afiliados', data_ati['CEDULA_COD'].nunique()) # 442570
print('Número de cedulas de los pensionistas', vej['CEDULA_COD'].nunique()) #627478

Número de cedulas de los afiliados 442570
Número de cedulas de los pensionistas 627478


In [16]:
ced_afi_vej = vej[vej['CEDULA_COD'].isin(data_ati['CEDULA_COD'].unique())]['CEDULA_COD'].unique() #401657

In [88]:
data_ati.head(10)

,CEDULA_COD,ANIO,MES,SALARIO,SALARIO_SECTOR,SECTOR_A,NUM_SEC_MES,%_NUM_SECTOR,GRUPO,SAL_PROM_GRUPO,...,SAL_PROM1,LS2,SAL_PROM2,LS_MS,ATI_CJ,ATI_KM,ATI_DB,BCS_CJ,BCS_KM,BCS_DB
0,126,2004,12,147.45,147.45,PRI,1,1,7,147.45,...,201.720411,382.495,201.720411,370.0,NaN,NaN,NaN,210.635,210.635,210.635
1,126,2005,1,161.67,161.67,PRI,1,1,6,161.67,...,201.720411,382.495,201.720411,370.0,NaN,NaN,NaN,210.635,210.635,210.635
2,126,2005,2,161.67,161.67,PRI,1,1,6,161.67,...,201.720411,382.495,201.720411,370.0,NaN,NaN,NaN,210.635,210.635,210.635
3,126,2005,3,161.67,161.67,PRI,1,1,6,161.67,...,201.720411,382.495,201.720411,370.0,NaN,NaN,NaN,210.635,210.635,210.635
4,126,2005,4,161.67,161.67,PRI,1,1,6,161.67,...,201.720411,382.495,201.720411,370.0,NaN,NaN,NaN,210.635,210.635,210.635
5,126,2005,5,161.67,161.67,PRI,1,1,6,161.67,...,201.720411,382.495,201.720411,370.0,NaN,NaN,NaN,210.635,210.635,210.635
6,126,2005,6,161.67,161.67,PRI,1,1,6,161.67,...,201.720411,382.495,201.720411,370.0,NaN,NaN,NaN,210.635,210.635,210.635
7,126,2005,7,161.67,161.67,PRI,1,1,6,161.67,...,201.720411,382.495,201.720411,370.0,NaN,NaN,NaN,210.635,210.635,210.635
8,126,2005,8,161.67,161.67,PRI,1,1,6,161.67,...,201.720411,382.495,201.720411,370.0,NaN,NaN,NaN,210.635,210.635,210.635
9,126,2005,9,161.67,161.67,PRI,1,1,6,161.67,...,201.720411,382.495,201.720411,370.0,NaN,NaN,NaN,210.635,210.635,210.635


In [94]:
val = ['CEDULA_COD','INI_CAL', 'FIN_CAL', 'BASE_CAL','LS2', 'SAL_PROM2', 'LS_MS', 'BCS_CJ','BCS_KM','BCS_DB', 'CJ', 'KM', 'DB']

data_ati['CJ'] = data_ati['CEDULA_COD'].map( data_ati.groupby('CEDULA_COD')['ATI_CJ'].agg(lambda x: x.max(skipna=True)) )
data_ati['KM'] = data_ati['CEDULA_COD'].map( data_ati.groupby('CEDULA_COD')['ATI_KM'].agg(lambda x: x.max(skipna=True)) )
data_ati['DB'] = data_ati['CEDULA_COD'].map( data_ati.groupby('CEDULA_COD')['ATI_DB'].agg(lambda x: x.max(skipna=True)) )

fir_data = data_ati.groupby('CEDULA_COD').first().reset_index()
fir_data1 = fir_data[val]



In [95]:
vej = pd.merge( vej, fir_data1, on='CEDULA_COD', how='left')

In [80]:
vej.head(50)

,CEDULA_COD,SEXO,FECHA_NACIMIENTO,FECHA_MUERTE,FECHA_DERECHO,NUMERO_IMPOSICIONES,COEFICIENTE_REAL,COEFICIENTE_CAL,PROMEDIO_SUELDO_REAL,PROMEDIO_CAL,...,PRES,INI_CAL,FIN_CAL,BASE_CAL,LS2,SAL_PROM2,LS_MS,BCS_CJ,BCS_KM,BCS_DB
0,114,H,1921-07-12,NaT,1994-07-01,429,0.500000,0.8125,0.00,144.9956,...,vej,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,115,H,1940-12-18,NaT,2006-05-01,368,0.758334,0.7500,0.00,364.9616,...,vej,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,117,H,1940-09-01,NaT,2006-03-01,183,0.500000,0.5625,0.00,67.8673,...,vej,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,119,M,1941-06-30,NaT,1997-08-01,421,0.814167,0.8125,0.00,14.4463,...,vej,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,121,M,1924-01-13,2022-01-27,2003-03-01,120,0.500000,0.5000,0.00,40.1640,...,vej,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,123,M,1942-07-07,NaT,2008-10-01,181,0.562500,0.5625,152.23,152.2296,...,vej,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,124,M,1931-06-26,2015-06-12,1990-11-01,120,0.500000,0.5000,0.00,2.6679,...,vej,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,126,M,1939-01-13,NaT,2011-01-01,288,0.675000,0.6750,213.24,210.6350,...,vej,2010-12-01,2006-01-01,210.635000,382.495,201.720411,370.00000,210.635000,210.635000,210.635000
8,127,M,1916-11-10,2018-03-25,1974-01-01,429,0.827500,0.8125,0.00,NaN,...,vej,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,134,M,1925-01-06,2023-07-13,1987-10-01,327,0.715625,0.7125,0.00,0.3114,...,vej,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
#Lectura de los pensionistas de invalidez
inv = pd.read_csv( ruta_inv, delimiter='\t', encoding='utf-8', decimal='.' )
inv = inv.sort_values( by=["CEDULA_COD"], ascending=[True] )
inv['FECHA_NACIMIENTO'] = pd.to_datetime(vej['FECHA_NACIMIENTO'], format='%d/%m/%Y')
inv['FECHA_MUERTE'] = pd.to_datetime(vej['FECHA_MUERTE'], format='%d/%m/%Y')
inv['FECHA_DERECHO'] = pd.to_datetime(vej['FECHA_DERECHO'], format='%d/%m/%Y')
inv['RANGO_INI_5MEJ'] = pd.to_datetime(vej['RANGO_INI_5MEJ'], format='%d/%m/%Y')
inv['RANGO_FIN_5MEJ'] = pd.to_datetime(vej['RANGO_FIN_5MEJ'], format='%d/%m/%Y')
inv['PRES']='inv'

In [62]:
inv

,CEDULA_COD,SEXO,FECHA_NACIMIENTO,FECHA_MUERTE,FECHA_DERECHO,NUMERO_IMPOSICIONES,COEFICIENTE_REAL,COEFICIENTE_CAL,PROMEDIO_SUELDO_REAL,PROMEDIO_CAL,VALOR_PENSION,PENSION_CAL,ID_PRESTACION,RANGO_INI_5MEJ,RANGO_FIN_5MEJ,N_MESES,PRES
42083,202,H,1951-08-16,NaT,2015-09-01,141,0.521875,0.5125,0.00,NaN,25.69,NaN,1368242,2015-08-31,2008-09-01,NaN,inv
11829,220,H,1926-02-08,2020-09-08,1982-11-01,161,0.437500,0.5375,0.00,NaN,404.72,NaN,1349069,1982-10-31,1977-11-01,NaN,inv
1,491,H,1940-12-18,NaT,2006-05-01,253,0.638542,0.6375,0.00,9.7173,27.77,6.194779,1385506,2006-04-30,2001-05-01,259.0,inv
5977,790,M,1947-12-20,NaT,2013-01-01,253,0.637500,0.6375,418.08,415.7311,266.53,265.028576,2456202,2012-12-31,2008-01-01,255.0,inv
42098,1019,H,1950-06-30,NaT,1996-11-01,60,0.437500,0.4375,0.00,0.2358,25.61,0.103163,1276408,1997-01-31,1992-02-01,18.0,inv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29866,23402574,H,1960-01-25,NaT,2022-02-01,109,0.487500,0.4875,408.93,424.3439,199.35,206.867651,2708508,2022-01-31,2012-02-01,110.0,inv
29867,23402574,H,1942-05-26,NaT,2007-07-01,109,0.000000,0.4875,0.00,424.3439,199.35,206.867651,3302141,2007-06-30,2002-07-01,110.0,inv
17812,23456125,H,1944-02-27,NaT,2015-10-01,68,0.437500,0.4375,574.18,578.5854,251.20,253.131113,3133014,2015-09-30,2010-10-01,69.0,inv
17825,23723908,NaN,1949-03-10,NaT,2009-10-01,74,0.450000,0.4500,295.12,NaN,170.00,NaN,2321091,2009-09-30,2004-10-01,NaN,inv


In [63]:
print('Número de cedulas de los pensionistas', inv['CEDULA_COD'].nunique()) #44815

Número de cedulas de los pensionistas 44815


In [64]:
print(inv[inv['CEDULA_COD'].isin(data_ati['CEDULA_COD'].unique())]['CEDULA_COD'].nunique()) #27339

27339


In [97]:
inv = pd.merge( inv, fir_data1, on='CEDULA_COD', how='left')

In [98]:
#Lectura de los pensionistas de discapacidad
dis = pd.read_csv( ruta_dis, delimiter='\t', encoding='utf-8', decimal='.' )
dis = dis.sort_values( by=["CEDULA_COD"], ascending=[True] )
dis['FECHA_NACIMIENTO'] = pd.to_datetime(vej['FECHA_NACIMIENTO'], format='%d/%m/%Y')
dis['FECHA_MUERTE'] = pd.to_datetime(vej['FECHA_MUERTE'], format='%d/%m/%Y')
dis['FECHA_DERECHO'] = pd.to_datetime(vej['FECHA_DERECHO'], format='%d/%m/%Y')
dis['RANGO_INI_5MEJ'] = pd.to_datetime(vej['RANGO_INI_5MEJ'], format='%d/%m/%Y')
dis['RANGO_FIN_5MEJ'] = pd.to_datetime(vej['RANGO_FIN_5MEJ'], format='%d/%m/%Y')
dis['PRES']='dis'

In [67]:
dis

,CEDULA_COD,SEXO,FECHA_NACIMIENTO,FECHA_MUERTE,FECHA_DERECHO,NUMERO_IMPOSICIONES,COEFICIENTE_REAL,COEFICIENTE_CAL,PROMEDIO_SUELDO_REAL,PROMEDIO_CAL,VALOR_PENSION,PENSION_CAL,ID_PRESTACION,RANGO_INI_5MEJ,RANGO_FIN_5MEJ,N_MESES,PRES
9163,17997,H,1947-01-15,NaT,2010-10-01,309,0.6875,0.6875,1677.05,2203.2088,1152.97,1514.706050,2583222,2010-09-30,2005-10-01,350,dis
11004,17997,H,1930-07-15,NaT,1976-03-01,309,0.0000,0.6875,0.00,2203.2088,1242.00,1514.706050,3219136,NaT,NaT,350,dis
9164,20394,H,1943-12-13,2014-07-23,1997-07-01,324,0.6875,0.7125,2551.71,2555.2520,1576.00,1820.617050,2752451,1997-06-30,1992-07-01,324,dis
11007,20701,H,1933-01-24,2022-08-15,1998-02-01,354,0.6875,0.7375,2744.63,2734.6833,1464.00,2016.828934,2471973,1998-01-31,1993-02-01,359,dis
3659,22909,H,1956-01-13,NaT,2021-02-01,352,0.6875,0.7375,630.92,628.2553,433.76,463.338284,2445574,2021-01-31,2016-02-01,358,dis
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11001,20831439,H,1944-03-23,2009-11-24,2009-03-23,302,0.6875,0.6875,1362.12,1363.1663,936.46,937.176831,3200672,2009-02-28,2004-03-01,312,dis
14661,20831581,H,1945-04-29,2011-12-17,2006-07-01,290,0.6875,0.6750,1728.46,2018.4658,1188.32,1362.464415,2825789,2006-06-30,2001-07-01,297,dis
5501,20847366,M,1961-06-03,NaT,2021-06-03,313,0.6875,0.7000,411.79,411.7910,297.50,288.253700,3191216,2020-05-31,2015-06-01,315,dis
14660,20856907,H,1945-10-06,NaT,2015-10-06,338,0.6875,0.7250,1376.48,1378.0005,946.33,999.050363,2944805,2015-09-30,1978-02-01,347,dis


In [69]:
print('Número de cedulas de los pensionistas', dis['CEDULA_COD'].nunique()) #14186

Número de cedulas de los pensionistas 14186


In [70]:
print(dis[dis['CEDULA_COD'].isin(data_ati['CEDULA_COD'].unique())]['CEDULA_COD'].nunique()) #13983

13983


In [99]:
dis = pd.merge( dis, fir_data1, on='CEDULA_COD', how='left')

In [100]:
pen = pd.concat( [ vej, inv, dis], axis=0)

In [73]:
pen

,CEDULA_COD,SEXO,FECHA_NACIMIENTO,FECHA_MUERTE,FECHA_DERECHO,NUMERO_IMPOSICIONES,COEFICIENTE_REAL,COEFICIENTE_CAL,PROMEDIO_SUELDO_REAL,PROMEDIO_CAL,...,PRES,INI_CAL,FIN_CAL,BASE_CAL,LS2,SAL_PROM2,LS_MS,BCS_CJ,BCS_KM,BCS_DB
0,114,H,1921-07-12,NaT,1994-07-01,429,0.500000,0.8125,0.00,144.9956,...,vej,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,115,H,1940-12-18,NaT,2006-05-01,368,0.758334,0.7500,0.00,364.9616,...,vej,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,117,H,1940-09-01,NaT,2006-03-01,183,0.500000,0.5625,0.00,67.8673,...,vej,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,119,M,1941-06-30,NaT,1997-08-01,421,0.814167,0.8125,0.00,14.4463,...,vej,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,121,M,1924-01-13,2022-01-27,2003-03-01,120,0.500000,0.5000,0.00,40.1640,...,vej,NaT,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14657,20831439,H,1944-03-23,2009-11-24,2009-03-23,302,0.687500,0.6875,1362.12,1363.1663,...,dis,2014-09-01,2009-10-01,1363.166333,2256.0800,908.063228,2048.56875,1351.016610,1351.016610,1351.016610
14658,20831581,H,1945-04-29,2011-12-17,2006-07-01,290,0.687500,0.6750,1728.46,2018.4658,...,dis,2020-02-01,2013-10-01,2018.465833,2525.5875,1415.478492,2187.69375,1662.981404,1662.981404,1662.981404
14659,20847366,M,1961-06-03,NaT,2021-06-03,313,0.687500,0.7000,411.79,411.7910,...,dis,2022-06-01,2017-07-01,411.791000,671.5000,313.183208,421.00000,410.323333,410.323333,410.323333
14660,20856907,H,1945-10-06,NaT,2015-10-06,338,0.687500,0.7250,1376.48,1378.0005,...,dis,2020-12-01,2015-01-01,1378.000500,2662.5000,981.761576,1350.00000,1350.000000,1350.000000,1350.000000


In [101]:
pen.to_csv('muestra_pensionistas.txt', sep='\t', index=False)

In [74]:
print('Cedulas que no cruzaron', pen[~pen['BASE_CAL'].isna()]['CEDULA_COD'].nunique()) #243476 cedulas que no cruzan

Cedulas que no cruzaron 442570


In [86]:
pen = pen[~pen['BASE_CAL'].isna()]

In [102]:
pen

,CEDULA_COD,SEXO,FECHA_NACIMIENTO,FECHA_MUERTE,FECHA_DERECHO,NUMERO_IMPOSICIONES,COEFICIENTE_REAL,COEFICIENTE_CAL,PROMEDIO_SUELDO_REAL,PROMEDIO_CAL,...,BASE_CAL,LS2,SAL_PROM2,LS_MS,BCS_CJ,BCS_KM,BCS_DB,CJ,KM,DB
0,114,H,1921-07-12,NaT,1994-07-01,429,0.500000,0.8125,0.00,144.9956,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,115,H,1940-12-18,NaT,2006-05-01,368,0.758334,0.7500,0.00,364.9616,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,117,H,1940-09-01,NaT,2006-03-01,183,0.500000,0.5625,0.00,67.8673,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,119,M,1941-06-30,NaT,1997-08-01,421,0.814167,0.8125,0.00,14.4463,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,121,M,1924-01-13,2022-01-27,2003-03-01,120,0.500000,0.5000,0.00,40.1640,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14657,20831439,H,1944-03-23,2009-11-24,2009-03-23,302,0.687500,0.6875,1362.12,1363.1663,...,1363.166333,2256.0800,908.063228,2048.56875,1351.016610,1351.016610,1351.016610,1.0,1.0,1.0
14658,20831581,H,1945-04-29,2011-12-17,2006-07-01,290,0.687500,0.6750,1728.46,2018.4658,...,2018.465833,2525.5875,1415.478492,2187.69375,1662.981404,1662.981404,1662.981404,1.0,1.0,1.0
14659,20847366,M,1961-06-03,NaT,2021-06-03,313,0.687500,0.7000,411.79,411.7910,...,411.791000,671.5000,313.183208,421.00000,410.323333,410.323333,410.323333,1.0,1.0,1.0
14660,20856907,H,1945-10-06,NaT,2015-10-06,338,0.687500,0.7250,1376.48,1378.0005,...,1378.000500,2662.5000,981.761576,1350.00000,1350.000000,1350.000000,1350.000000,1.0,1.0,1.0
